In [135]:
import pandas as pd

chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding='ISO-8859-1')

### Computing Missing Values in Chunks

In [136]:
empty_values = []

for chunk in chunk_iter:
    empty_values.append(chunk.isnull().sum())

empty_values = pd.concat(empty_values)

empty_values = empty_values.groupby(empty_values.index).sum().reset_index().rename(columns={"index":"col_name", 0:"MissingValue"})

empty_values.sort_values(by="MissingValue", ascending=False)



,col_name,MissingValue
12,investor_category_code,50427
18,investor_state_code,16809
13,investor_city,12480
14,investor_country_code,12001
19,raised_amount_usd,3599
0,company_category_code,643
1,company_city,533
6,company_state_code,492
7,funded_at,3
8,funded_month,3


### Memory Usage in MB for each Column

In [137]:
memory = []
chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding='ISO-8859-1')

def memory_usage(df):
    mem = df.memory_usage(deep=True) / (1024*1024)
    return mem

for chunk in chunk_iter:
    chunk = memory_usage(chunk)
    memory.append(chunk)

memory_summary = pd.concat(memory)
memory_summary = memory_summary.groupby(memory_summary.index).sum().reset_index().rename(columns={"index":"col_name", 0:"Memory in MB"})
memory_summary.sort_values(by="Memory in MB", ascending=False)


,col_name,Memory in MB
17,investor_permalink,4.749821
5,company_permalink,3.869808
16,investor_name,3.734270
4,company_name,3.424955
8,funded_at,3.378091
2,company_city,3.343473
1,company_category_code,3.262619
6,company_region,3.253503
12,funding_round_type,3.252704
18,investor_region,3.238946


### Total Memory in All Chunks

In [138]:
memory_summary.sum()

col_name        Indexcompany_category_codecompany_citycompany_...
Memory in MB                                            56.988912
dtype: object

### Drop Columns Not Useful for Analysis

Columns that contain the word "link" and those with more than 90% missing values.

1. Calculate first the length of the dataframe
2. Get a percentage

In [139]:
remove_cols = memory_summary[memory_summary["col_name"].str.contains("link")]["col_name"].to_list()
remove_cols

['company_permalink', 'investor_permalink']

In [140]:
length = 0
chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding='ISO-8859-1')

for chunk in chunk_iter:
    length += len(chunk)

empty_values["Pct%_Missing"] = (empty_values["MissingValue"] / length) * 100

empty_values.sort_values(by="Pct%_Missing", ascending=False)

,col_name,MissingValue,Pct%_Missing
12,investor_category_code,50427,95.379232
18,investor_state_code,16809,31.793077
13,investor_city,12480,23.605069
14,investor_country_code,12001,22.699073
19,raised_amount_usd,3599,6.807263
0,company_category_code,643,1.216191
1,company_city,533,1.008133
6,company_state_code,492,0.930584
7,funded_at,3,0.005674
8,funded_month,3,0.005674


In [141]:
#remove_cols.append("investor_category_code")


remove_cols

['company_permalink', 'investor_permalink']

In [142]:
cols = pd.read_csv("crunchbase-investments.csv", encoding='ISO-8859-1', nrows=2).columns

keep_cols = []

for col in cols:
    if col not in remove_cols:
        keep_cols.append(col)
        
keep_cols

['company_name',
 'company_category_code',
 'company_country_code',
 'company_state_code',
 'company_region',
 'company_city',
 'investor_name',
 'investor_category_code',
 'investor_country_code',
 'investor_state_code',
 'investor_region',
 'investor_city',
 'funding_round_type',
 'funded_at',
 'funded_month',
 'funded_quarter',
 'funded_year',
 'raised_amount_usd']

### Selecting the Most Efficient Data Type

In [143]:
chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

d_types = []
count = 0

for chunk in chunk_iter:
        chunk = chunk.dtypes.to_frame().reset_index().rename(columns={"index":"col_name", 0:"datatype"})
        d_types.append(chunk)

d_types = pd.concat(d_types).drop_duplicates(subset=["col_name", "datatype"])

d_types[d_types.duplicated(subset=["col_name"], keep=False)].sort_values(by="col_name")

,col_name,datatype
16,funded_year,int64
16,funded_year,float64
7,investor_category_code,object
7,investor_category_code,float64
11,investor_city,object
11,investor_city,float64
8,investor_country_code,object
8,investor_country_code,float64
9,investor_state_code,object
9,investor_state_code,float64


In [144]:
few_rows = pd.read_csv("crunchbase-investments.csv", nrows=100, encoding='ISO-8859-1', usecols=keep_cols)
check = d_types[d_types.duplicated(subset=["col_name"], keep=False)].sort_values(by="col_name")["col_name"].unique().tolist()
few_rows[check]

,funded_year,investor_category_code,investor_city,investor_country_code,investor_state_code
0,2012,NaN,New York,USA,NY
1,2012,finance,Columbus,USA,OH
2,2012,finance,Columbus,USA,OH
3,2012,finance,Columbus,USA,OH
4,2011,finance,Columbus,USA,OH
...,...,...,...,...,...
95,2010,public_relations,Boston,USA,MA
96,2009,public_relations,Boston,USA,MA
97,2012,NaN,NaN,NaN,NaN
98,2008,biotech,Stirling,GBR,NaN


### Loading Chunks into SQLite

In [145]:
import sqlite3

chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

conn = sqlite3.connect("crunchbase.db")

for chunk in chunk_iter:
    chunk.to_sql("investment", conn, if_exists="append", index=None)

What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.

In [146]:
statement = "SELECT raised_amount_usd FROM investment ORDER BY raised_amount_usd DESC;"

proportions = pd.read_sql(statement, conn)

total = proportions.sum()

top_10 = proportions[0:int(len(proportions) * 0.10)].sum() / total
top_1 = proportions[0:int(len(proportions) * 0.01)].sum() / total

print(top_10, "\n", top_1)

print("Top 10% raised 51% and Top 1% raised 20% of investments")

raised_amount_usd    0.514357
dtype: float64 
 raised_amount_usd    0.198236
dtype: float64
Top 10% raised 51% and Top 1% raised 20% of investments


In [147]:
bottom_10 = (proportions[-int(len(proportions) * 0.10):].sum() / total) * 100
bottom_1 = (proportions[-int(len(proportions) * 0.01):].sum() / total) * 100

print(bottom_10, "\n", bottom_1)

print("Bottom 10% raised 0.02% and Bottom 1% raised 0.00% of investments")

raised_amount_usd    0.022217
dtype: float64 
 raised_amount_usd    0.0
dtype: float64
Bottom 10% raised 0.02% and Bottom 1% raised 0.00% of investments


Which category of company attracted the most investments?

In [148]:
statement2 = "SELECT company_category_code, SUM(raised_amount_usd) as total_investment FROM investment GROUP BY company_category_code ORDER BY total_investment DESC"

category = pd.read_sql(statement2, conn)

category.head(15)

,company_category_code,total_investment
0,biotech,1.103964e+11
1,software,7.308452e+10
2,mobile,6.477738e+10
3,cleantech,5.270523e+10
4,enterprise,4.586093e+10
5,web,4.014326e+10
6,medical,2.536711e+10
7,advertising,2.507666e+10
8,ecommerce,2.256722e+10
9,network_hosting,2.241968e+10


Which investor contributed the most money (across all startups)?

In [149]:
statement3 = "SELECT investor_name, SUM(raised_amount_usd) as total_raised FROM investment GROUP BY investor_name ORDER BY total_raised DESC"

investor = pd.read_sql(statement3, conn)

investor

,investor_name,total_raised
0,Kleiner Perkins Caufield & Byers,1.121783e+10
1,New Enterprise Associates,9.692542e+09
2,Accel Partners,6.472126e+09
3,Goldman Sachs,6.375459e+09
4,Sequoia Capital,6.039402e+09
...,...,...
10461,313 Ventures,NaN
10462,2x Consumer Products Growth Partners,NaN
10463,212 Capital Partners,NaN
10464,1in10 Ventures,NaN


Which investors contributed the most money per startup?

In [150]:
statement4 = "SELECT investor_name, AVG(raised_amount_usd) as avg_contribution FROM investment GROUP BY investor_name"

contributions = pd.read_sql(statement4, conn)

contributions.groupby(["investor_name"]).mean().sort_values(by="avg_contribution", ascending=False).head(20)

,avg_contribution
investor_name,
Marlin Equity Partners,2.600000e+09
BrightHouse,2.350000e+09
GI Partners,1.050000e+09
Sprint Nextel,8.333333e+08
Siemens PLM Software,7.500000e+08
Comcast,6.298889e+08
Eagle River Holdings,6.142500e+08
Time Warner,5.209091e+08
Laurel Crown Partners,4.500000e+08


Which funding round was the most popular? Which was the least popular?

In [151]:
statement5 = "SELECT funding_round_type, COUNT(*) as freq FROM investment GROUP BY funding_round_type"

funding_rounds = pd.read_sql(statement5, conn)

funding_rounds.sort_values(by="freq", ascending=False)

,funding_round_type,freq
6,series-a,13938
8,series-c+,10870
1,angel,8989
9,venture,8917
7,series-b,8794
3,other,964
5,private-equity,357
4,post-ipo,33
2,crowdfunding,5
0,None,3


1. Understand which columns the data sets share, and how the data sets are linked.
2. Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.
3. Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [152]:
files = ["crunchbase-companies.csv", "crunchbase-rounds.csv", "crunchbase-acquisitions.csv"]
names = ["companies", "rounds", "acquisitions"]

for file, name in zip(files, names):
    chunk_iter = pd.read_csv(file, chunksize=5000, encoding='ISO-8859-1')
    for chunk in chunk_iter:
        chunk.to_sql(name, conn, if_exists="append")